In [1]:
import requests

In [2]:
path_params = {
    'assetclass': 'etf'
}

headers = {
    'Accept': 'application/json,text/plain,*/*',
    'Cache-Control': 'no-cache',
    'Dnt': '1',
    'Origin': 'https://www.nasdaq.com',
    'Referer': 'https://www.nasdaq.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51'
}

r = requests.get('https://api.nasdaq.com/api/quote/SPY/info', params=path_params, headers=headers, timeout=5)
print(r.json())

{'data': {'symbol': 'SPY', 'companyName': 'SPDR S&P 500 ETF Trust Units', 'stockType': None, 'exchange': 'PSE', 'isNasdaqListed': False, 'isNasdaq100': False, 'isHeld': False, 'primaryData': {'lastSalePrice': '$431.605', 'netChange': '+0.215', 'percentageChange': '+0.05%', 'deltaIndicator': 'up', 'lastTradeTimestamp': 'Sep 22, 2023 1:51 PM ET', 'isRealTime': True, 'bidPrice': '$431.60', 'askPrice': '$431.61', 'bidSize': '700', 'askSize': '607', 'volume': '45,004,078'}, 'secondaryData': None, 'marketStatus': 'Open', 'assetClass': 'ETF', 'keyStats': None, 'notifications': []}, 'message': None, 'status': {'rCode': 200, 'bCodeMessage': None, 'developerMessage': None}}


In [3]:
def retrieve_historical(ticker: str) -> dict:
    path_params = {
        'assetclass': 'etf',
        'fromdate': '2013-06-20',
        'todate': '2023-09-03',
        'limit': '9999'
    }

    headers = {
        'Accept': 'application/json,text/plain,*/*',
        'Cache-Control': 'no-cache',
        'Dnt': '1',
        'Origin': 'https://www.nasdaq.com',
        'Referer': 'https://www.nasdaq.com',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36 Edg/114.0.1823.51'
    }
    
    return requests.get(f'https://api.nasdaq.com/api/quote/{ticker}/historical', params=path_params, headers=headers, timeout=5).json()

In [4]:
retrieve_historical('SPY')

{'data': {'symbol': 'SPY',
  'totalRecords': 2504,
  'tradesTable': {'asOf': None,
   'headers': {'date': 'Date',
    'close': 'Close/Last',
    'volume': 'Volume',
    'open': 'Open',
    'high': 'High',
    'low': 'Low'},
   'rows': [{'date': '09/01/2023',
     'close': '451.19',
     'volume': '58,944,100',
     'open': '453.17',
     'high': '453.67',
     'low': '449.68'},
    {'date': '08/31/2023',
     'close': '450.35',
     'volume': '66,084,560',
     'open': '451.65',
     'high': '452.83',
     'low': '450.16'},
    {'date': '08/30/2023',
     'close': '451.01',
     'volume': '69,053,910',
     'open': '449.51',
     'high': '451.67',
     'low': '448.78'},
    {'date': '08/29/2023',
     'close': '449.16',
     'volume': '83,081,920',
     'open': '442.65',
     'high': '449.45',
     'low': '442.46'},
    {'date': '08/28/2023',
     'close': '442.76',
     'volume': '61,595,390',
     'open': '442.24',
     'high': '443.40',
     'low': '439.9728'},
    {'date': '08/25/2

In [9]:
import time
from datetime import datetime

historical_data = {}
unread_tickers = []

history = retrieve_historical("SPY")
if (history is not None and history['data'] is not None
        and history['data']['tradesTable'] is not None
        and history['data']['tradesTable']['rows']):
    history = history['data']['tradesTable']['rows']
    
    pe_cursor = 0
    history.reverse()
    for i in range(0, len(history)):
        history[i]['date'] = datetime.strptime(history[i]['date'],"%m/%d/%Y").timestamp()
        history[i]['open'] = float(history[i]['open'].replace(',', ''))
        history[i]['close'] = float(history[i]['close'].replace(',', ''))
        history[i]['high'] = float(history[i]['high'].replace(',', ''))
        history[i]['low'] = float(history[i]['low'].replace(',', ''))
        if history[i]['volume'] != 'N/A':
            history[i]['volume'] = int(history[i]['volume'].replace(',', ''))
        else:
            history[i]['volume'] = None
    historical_data["SPY"] = history

    print('Download complete!!!')
else:
    print('Ticker was not readable')

Download complete!!!


In [10]:
import json

with open("spy_stock_history.json", "w") as outfile:
    json.dump(historical_data, outfile)